In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [3]:
# 데이터 불러오기 및 전처리 작업
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=16, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader = DataLoader(testset, batch_size=16,shuffle=False)

# Class
#'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.DEFAULT)

In [6]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [7]:
num_ftrs = model.classifier[6].in_features
model.features[0] = nn.Conv2d(3, 64, kernel_size=5, stride=1)
model.classifier[6] = nn.Linear(in_features=num_ftrs, out_features=10)
model = model.to(device)

In [8]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inpla

In [10]:
i = 0
for i, (name, param) in enumerate(model.named_parameters()):
    print(i, name)
    

0 features.0.weight
1 features.0.bias
2 features.3.weight
3 features.3.bias
4 features.6.weight
5 features.6.bias
6 features.8.weight
7 features.8.bias
8 features.10.weight
9 features.10.bias
10 classifier.1.weight
11 classifier.1.bias
12 classifier.4.weight
13 classifier.4.bias
14 classifier.6.weight
15 classifier.6.bias


In [11]:
for i, (name, param) in enumerate(model.named_parameters()):
    param.requires_grad = False
    if i == 9:
        print(f'ended in {i}th param')
        break

ended in 9th param


In [15]:
for i, (name, param) in enumerate(model.named_parameters()):
    
    print(name, param.requires_grad)

features.0.weight False
features.0.bias False
features.3.weight False
features.3.bias False
features.6.weight False
features.6.bias False
features.8.weight False
features.8.bias False
features.10.weight False
features.10.bias False
classifier.1.weight True
classifier.1.bias True
classifier.4.weight True
classifier.4.bias True
classifier.6.weight True
classifier.6.bias True


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

In [18]:
epochs = 20
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    cost = running_loss / len(train_loader)
    print(f'{epoch+1} loss: {cost:.3f}')

print('Finished Training')

1 loss: 1.870
2 loss: 1.852
3 loss: 1.843
4 loss: 1.834
5 loss: 1.831
6 loss: 1.828
7 loss: 1.824
8 loss: 1.821
9 loss: 1.819
10 loss: 1.821
11 loss: 1.824
12 loss: 1.816
13 loss: 1.819
14 loss: 1.818
15 loss: 1.815
16 loss: 1.812
17 loss: 1.812
18 loss: 1.815
19 loss: 1.813
20 loss: 1.810
Finished Training


In [20]:
correct = 0
total = 0
with torch.inference_mode():
    model.eval()
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        total += labels.size(0)
        correct += (preds.argmax(axis=1) == labels).sum().item()
    
print(f'Accuracy of the network on the 10,000 test images: {100 * correct / total}')

Accuracy of the network on the 10,000 test images: 39.05
